# 维基百科爬虫

In [4]:
import requests
from scrapy.selector import Selector
from lxml import etree
from concurrent.futures import ThreadPoolExecutor

ModuleNotFoundError: No module named 'scrapy'

In [11]:
def crawl_wiki(url): 
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
        # 设置简体中文
        'Accept-Language': 'zh-CN,zh'
    }
        
    with open(f'../data/wiki.csv', 'a', encoding='utf-8') as f:
        response = requests.get(url, headers=headers)

        html = etree.HTML(response.text)

        # 删除所有的style标签及其内容
        etree.strip_elements(html,'style', 'sup', with_tail=False)

        # 创建一个新的变量response_text，并将response.text的值赋给它
        response_text = etree.tostring(html, encoding='utf-8').decode('utf-8')

        selector = Selector(text=response_text)

        # 提取文本
        sentence = selector.xpath('//*[@id="mw-content-text"]//p//text()').extract()

        # 移除换行符和多余的空格
        sentence = [s.replace('\n', '').strip() for s in sentence]

        # 连接文本
        join_sentence = ''.join(sentence)

        # 写入文件
        f.write(join_sentence + '\n')

        # print(join_sentence)

In [12]:
def main():
    # 维基百科随机词条url
    url = 'https://zh.wikipedia.org/wiki/Special:%E9%9A%8F%E6%9C%BA%E9%A1%B5%E9%9D%A2'

    # 创建ThreadPoolExecutor，设置线程数量
    with ThreadPoolExecutor(max_workers=32) as executor:
        # 使用线程池并行爬取页面
        executor.map(crawl_wiki, [url] * 3)  # 将url放入一个列表中

In [13]:
if __name__ == '__main__':
    for i in range(100):
        main()